
---

# **Модуль 1. Введение в очереди сообщений и RabbitMQ**
---

## Цели сегодняшнего занятия:

1. Понять, **зачем нужны очереди сообщений** в современных системах.
2. Познакомиться с **RabbitMQ и протоколом AMQP**.
3. Установить RabbitMQ локально или через Docker.
4. Попробовать **веб-интерфейс управления** RabbitMQ.

---


## Часть 1: Зачем вообще нужны очереди сообщений?

Давайте представим простой сценарий:

У вас есть интернет-магазин. Пользователь оформляет заказ. При этом:

* Нужно отправить письмо о заказе
* Уменьшить количество товаров на складе
* Создать задачу для доставки
* Вызвать систему оплаты

Если все эти действия происходят **сразу в одном процессе**, то:

* Если одно из них упадёт, всё рушится
* Время отклика увеличивается
* Нет возможности масштабировать отдельно нужные части

**Очередь сообщений** позволяет нам **разделить** эти процессы: основной сервис просто кладёт сообщение в очередь, а специализированные worker'ы (консьюмеры) занимаются своей задачей.

Это называется **асинхронная архитектура**.

---



## Часть 2: Что такое RabbitMQ?

### Определение:

**RabbitMQ** — это брокер сообщений, реализующий протокол **AMQP (Advanced Message Queuing Protocol)**.

Он позволяет:

* Отправлять сообщения от одного компонента к другому
* Буферизовать сообщения, если получатель недоступен
* Надёжно хранить сообщения до подтверждения
* Распределять нагрузку между обработчиками

Используется такими компаниями, как Instagram, Slack, GitHub, WeChat, Shopify и др.

---


## Часть 3: Архитектура RabbitMQ (простыми словами)

Вот главные участники:

| Компонент    | Роль                                                          |
| ------------ | ------------------------------------------------------------- |
| **Producer** | Отправляет сообщение                                          |
| **Exchange** | Получает сообщение от Producer'а и решает, куда его отправить |
| **Queue**    | Хранит сообщения, пока Consumer не заберёт их                 |
| **Consumer** | Забирает и обрабатывает сообщение                             |

Аналогия:

* Producer — это повар
* Exchange — официант
* Queue — стол, на который кладут готовые блюда
* Consumer — клиент, который забирает и ест блюдо

---


## Часть 4: Установка RabbitMQ

### Вариант 1: Через Docker (рекомендуется)

```bash
docker run -d --name rabbitmq \
  -p 5672:5672 -p 15672:15672 \
  rabbitmq:3-management
```

* Порт **5672** — основной протокол AMQP
* Порт **15672** — панель управления (UI)

После запуска откройте браузер:
[http://localhost:15672](http://localhost:15672)
(логин: `guest`, пароль: `guest`)

---

### Вариант 2: Установка вручную

* Windows/macOS: [https://www.rabbitmq.com/download.html](https://www.rabbitmq.com/download.html)
* Ubuntu:

```bash
sudo apt update
sudo apt install rabbitmq-server
sudo systemctl enable rabbitmq-server
sudo systemctl start rabbitmq-server
sudo rabbitmq-plugins enable rabbitmq_management
```

---



## Часть 5: Первое знакомство с интерфейсом RabbitMQ

Перейдите на [http://localhost:15672](http://localhost:15672). Что вы видите:

* **Overview**: общее состояние брокера
* **Connections**: текущие соединения с клиентами
* **Channels**: каналы — логические соединения поверх TCP
* **Exchanges**: маршрутизаторы сообщений
* **Queues**: все очереди в системе
* **Admin**: пользователи, права, настройки

Создайте свою первую очередь:

1. Перейдите на вкладку **Queues**
2. Нажмите **Add a new queue**
3. Введите имя, например `my_first_queue`, нажмите Add queue
4. Перейдите на вкладку **Publish message**
5. Отправьте текстовое сообщение
6. Перейдите во вкладку **Get messages**, нажмите **Get Message(s)**

🎉 Поздравляю! Вы отправили и получили своё первое сообщение через RabbitMQ!

---


## Часть 6: Задание на практику

### Практическое задание:

1. Установите RabbitMQ (Docker или вручную).
2. Откройте интерфейс управления.
3. Создайте очередь `student_test_queue`.
4. Опубликуйте в неё сообщение "Hello, RabbitMQ!".
5. Получите сообщение через UI.
6. Попробуйте отправить несколько сообщений и получить их по одному.


## 🧠 Краткий итог занятия

| Что узнали                                                 | Что делали                     |
| ---------------------------------------------------------- | ------------------------------ |
| Что такое брокер сообщений и зачем он нужен                | Запустили RabbitMQ в Docker    |
| Архитектура RabbitMQ (Producer, Exchange, Queue, Consumer) | Зашли в UI и создали очередь   |
| Что такое AMQP и почему это важно                          | Отправили и получили сообщение |

---



---

# **Модуль 2. Основы AMQP: Очереди, Exchange и Роутинг сообщений**

---

## Цели занятия:

1. Понять роли **Producer**, **Exchange**, **Queue**, **Consumer**.
2. Изучить **типы exchange** и их поведение.
3. Разобраться с **Routing Key**, **Binding** и **Acknowledgements**.
4. Написать свой первый скрипт отправки и получения сообщений с использованием библиотеки **pika**.

---



## Часть 1: Архитектура AMQP в деталях

Повторим основные участники:

* **Producer** — отправляет сообщение
* **Exchange** — решает, в какую очередь отправить сообщение
* **Queue** — хранит сообщения до получения
* **Consumer** — получает сообщение

### Что нового?

* **Routing Key** — метка, по которой сообщение направляется в нужную очередь
* **Binding** — правило маршрутизации (связь между очередью и exchange)

---


## Часть 2: Типы Exchange

| Тип Exchange | Что делает                                                      | Пример                                                    |
| ------------ | --------------------------------------------------------------- | --------------------------------------------------------- |
| **Direct**   | Маршрутизирует по точному соответствию routing key              | `log.info` попадёт в очередь, подписанную на `log.info`   |
| **Fanout**   | Рассылает всем очередям, связанным с ним, игнорируя routing key | Отправка всем микросервисам сразу                         |
| **Topic**    | Поддерживает шаблоны (`*`, `#`)                                 | `log.*` — любые однословные логи; `log.#` — все вложенные |
| **Headers**  | Использует заголовки вместо routing key                         | Более гибкая маршрутизация                                |

---



## Часть 3: Установка библиотеки pika (Python)

```bash
pip install pika
```

---

## Часть 4: Пишем первый **Producer**

In [ ]:
# producer.py
import pika

connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

channel.queue_declare(queue='hello')

channel.basic_publish(
    exchange='',
    routing_key='hello',
    body='Hello RabbitMQ!'
)

print(" [x] Sent 'Hello RabbitMQ!'")
connection.close()


### Что происходит:

* Создаётся соединение и канал
* Объявляется очередь `hello`
* Отправляется сообщение в эту очередь напрямую (exchange = '')

---


## Часть 5: Пишем **Consumer**

In [ ]:
# consumer.py
import pika

def callback(ch, method, properties, body):
    print(f" [x] Received {body}")

connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

channel.queue_declare(queue='hello')

channel.basic_consume(
    queue='hello',
    on_message_callback=callback,
    auto_ack=True
)

print(' [*] Waiting for messages. To exit press CTRL+C')
channel.start_consuming()


### Что происходит:

* Объявляется очередь `hello`
* Настраивается функция обратного вызова (`callback`)
* `start_consuming()` запускает бесконечное ожидание новых сообщений

---

## Демонстрация: Как это работает

1. Откройте терминал и запустите `consumer.py`
2. В другом терминале запустите `producer.py`
3. Посмотрите, как сообщение появляется у Consumer’а

---


## Часть 6: Что такое Acknowledgement (ack)

По умолчанию RabbitMQ ожидает, что Consumer **подтвердит**, что он получил и обработал сообщение.

* `auto_ack=True` — сообщение сразу помечается как доставленное (ненадёжно)
* `auto_ack=False` — вы сами вызываете `ch.basic_ack(delivery_tag=method.delivery_tag)`

Это позволяет повторно отправить сообщение, если Consumer "упал".

---


## Часть 7: Практика — Direct Exchange

### Пример: разные уровни логов


In [ ]:
# direct_producer.py
channel.exchange_declare(exchange='logs_direct', exchange_type='direct')
channel.basic_publish(exchange='logs_direct', routing_key='info', body='Info log!')

In [ ]:
# direct_consumer.py
channel.exchange_declare(exchange='logs_direct', exchange_type='direct')
queue = channel.queue_declare('', exclusive=True).method.queue
channel.queue_bind(exchange='logs_direct', queue=queue, routing_key='info')

Вывод:

* Сообщение попадёт только в те очереди, которые подписаны на `info`

---


## 📂 Практическое задание:

1. Напишите Producer и Consumer для очереди `hello`
2. Измените их так, чтобы использовать Direct Exchange с ключами `info`, `warning`, `error`
3. Запустите 3 Consumer’а, каждый на разный ключ
4. Отправьте по 1 сообщению на каждый ключ и проверьте, кто их получил

---

| Поняли                                             | Попробовали                          |
| -------------------------------------------------- | ------------------------------------ |
| Как устроены Exchange, Queue, Binding, Routing Key | Написали producer и consumer         |
| В чём отличие direct и fanout exchange             | Отправили сообщение в очередь        |
| Как работает ack                                   | Запустили систему с обратным вызовом |

---


# **Модуль 3. Надёжность доставки и масштабирование RabbitMQ**
---

## Цели занятия:

1. Научиться обеспечивать **надёжную доставку сообщений**.
2. Разобраться с **подтверждениями** (ack), **устойчивыми очередями** и **персистентными сообщениями**.
3. Познакомиться с **prefetch count** и балансировкой нагрузки.
4. Понять, как RabbitMQ обрабатывает сбои и как использовать **Dead Letter Exchange** (DLX).
5. Обсудить варианты **масштабирования** — horizontal scaling и HA.

---


## Часть 1: Acknowledgements (подтверждения)

По умолчанию RabbitMQ считает, что сообщение доставлено успешно, **только после ack от consumer’а**.

### Два режима:

* `auto_ack=True` → ненадёжно, RabbitMQ удаляет сообщение сразу
* `auto_ack=False` + `ch.basic_ack()` → надёжно

### Пример:


In [ ]:
def callback(ch, method, properties, body):
    try:
        process_message(body)
        ch.basic_ack(delivery_tag=method.delivery_tag)
    except Exception:
        print("Ошибка обработки")


---

## Часть 2: Устойчивость (Durability)

### 1. Устойчивые очереди (durable)

```python
channel.queue_declare(queue='task_queue', durable=True)
```

> Очередь не исчезнет при перезапуске брокера

### 2. Персистентные сообщения

```python
channel.basic_publish(
    exchange='',
    routing_key='task_queue',
    body='My Task',
    properties=pika.BasicProperties(delivery_mode=2)  # persistent
)
```

> Только при **двойной защите**: очередь *и* сообщение должны быть устойчивыми

---


## Часть 3: Prefetch (QOS)

### Проблема:

Если у вас много consumer'ов, RabbitMQ может отправить сразу пачку сообщений одному и "засыпать" его.

### Решение:

```python
channel.basic_qos(prefetch_count=1)
```

> Позволяет отправлять максимум 1 сообщение на consumer, пока он не сделает ack.

Это позволяет:

* Балансировать нагрузку
* Исключить "захват" всех сообщений одним worker'ом

---


## Часть 4: Dead Letter Exchange (DLX)

Когда сообщение:

* Не обработано
* Было отвергнуто (reject)
* Просрочено (TTL)

...оно может быть направлено в **альтернативную очередь** для анализа или повторной обработки.

### Пример:

```python
channel.queue_declare(
    queue='main_queue',
    arguments={
        'x-dead-letter-exchange': 'dlx_exchange'
    }
)
```

> Это позволяет **отлавливать "плохие" сообщения**, а не терять их бесследно

---


## Часть 5: Масштабирование Consumer’ов

Когда сообщений становится много:

### Горизонтальное масштабирование:

* Запускаем несколько consumer-процессов (или контейнеров)
* RabbitMQ сам распределяет сообщения между ними (Round-robin)

```bash
# В разных терминалах:
python worker.py
python worker.py
```

Если вы используете prefetch=1, сообщения будут распределяться более честно (fair dispatch).

---


## Часть 6: Надёжная обработка задач (пример)

### Producer:

In [ ]:
channel.queue_declare(queue='task_queue', durable=True)
channel.basic_publish(
    exchange='',
    routing_key='task_queue',
    body='Task 1',
    properties=pika.BasicProperties(delivery_mode=2)
)

### Worker (Consumer):

In [ ]:
channel.queue_declare(queue='task_queue', durable=True)
channel.basic_qos(prefetch_count=1)

def callback(ch, method, properties, body):
    print(f"Получено: {body}")
    time.sleep(5)
    print("Готово")
    ch.basic_ack(delivery_tag=method.delivery_tag)

channel.basic_consume(queue='task_queue', on_message_callback=callback)


> Если worker завершится аварийно — сообщение вернётся в очередь.

---

## Практическое задание:

1. Создайте очередь `task_queue` с флагом `durable=True`.
2. Отправьте 5 сообщений с `delivery_mode=2`.
3. Напишите worker с `prefetch_count=1` и `manual ack`.
4. Запустите 2 worker’а и убедитесь, что сообщения делятся поровну.
5. Убейте одного worker'а во время обработки — убедитесь, что сообщение возвращается в очередь.
6. Добавьте DLX и проверьте, как отлавливаются ошибки.

---

## Вопросы для обсуждения:

* Что произойдёт, если consumer не отправит ack?
* Зачем нужен prefetch?
* Когда стоит использовать DLX?

---




## Краткий итог:

| Изучили                                | Почему это важно                                |
| -------------------------------------- | ----------------------------------------------- |
| Acknowledgements (ручной контроль ack) | Избегаем потери сообщений                       |
| Durable + Persistent                   | Гарантируем, что сообщения не исчезнут при сбое |
| Prefetch                               | Эффективное распределение нагрузки              |
| DLX                                    | Работа с "плохими" сообщениями                  |
| Масштабирование worker’ов              | Повышаем пропускную способность системы         |

---


---

# **Модуль 4. Расширенные паттерны и интеграция RabbitMQ**

---

## Цели занятия:

1. Реализовать паттерн **RPC (Remote Procedure Call)**.
2. Познакомиться с **Delayed Messaging** — отложенными сообщениями.
3. Освоить **Retry-паттерны**.
4. Научиться интегрировать RabbitMQ с **Flask/FastAPI**.
5. Поговорить о безопасности: аутентификация и SSL.

---



## Часть 1: RPC через RabbitMQ

### Зачем это нужно?

Иногда одного "огня в очередь" мало. Нужно получить **ответ**: например, посчитать что-то и вернуть результат.

**RPC-паттерн** позволяет имитировать вызов функции на другом сервисе.

---

### Принцип работы:

1. **Client**:

   * Отправляет запрос в очередь
   * Указывает `reply_to` — временную очередь для ответа
   * Ставит `correlation_id` — уникальный идентификатор запроса

2. **Server**:

   * Получает запрос
   * Обрабатывает
   * Отправляет результат в `reply_to` с тем же `correlation_id`

3. **Client**:

   * Ожидает в своей очереди
   * Сопоставляет ответ по `correlation_id`

---

### Пример на Python

#### RPC Server:


In [ ]:
# rpc_server.py
import pika

def on_request(ch, method, props, body):
    n = int(body)
    print(f" [.] fib({n})")
    response = fib(n)

    ch.basic_publish(
        exchange='',
        routing_key=props.reply_to,
        properties=pika.BasicProperties(
            correlation_id=props.correlation_id
        ),
        body=str(response)
    )
    ch.basic_ack(delivery_tag=method.delivery_tag)

def fib(n):
    if n == 0: return 0
    if n == 1: return 1
    return fib(n-1) + fib(n-2)

connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()
channel.queue_declare(queue='rpc_queue')

channel.basic_qos(prefetch_count=1)
channel.basic_consume(queue='rpc_queue', on_message_callback=on_request)

print(" [x] Awaiting RPC requests")
channel.start_consuming()

---

#### RPC Client:

In [ ]:
# rpc_client.py
import pika
import uuid

class FibonacciRpcClient:
    def __init__(self):
        self.connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
        self.channel = self.connection.channel()

        result = self.channel.queue_declare('', exclusive=True)
        self.callback_queue = result.method.queue

        self.channel.basic_consume(
            queue=self.callback_queue,
            on_message_callback=self.on_response,
            auto_ack=True
        )
        self.response = None
        self.corr_id = None

    def on_response(self, ch, method, props, body):
        if self.corr_id == props.correlation_id:
            self.response = body

    def call(self, n):
        self.response = None
        self.corr_id = str(uuid.uuid4())
        self.channel.basic_publish(
            exchange='',
            routing_key='rpc_queue',
            properties=pika.BasicProperties(
                reply_to=self.callback_queue,
                correlation_id=self.corr_id
            ),
            body=str(n)
        )
        while self.response is None:
            self.connection.process_data_events()
        return int(self.response)

fib_rpc = FibonacciRpcClient()
print(" [x] Requesting fib(10)")
response = fib_rpc.call(10)
print(f" [.] Got {response}")


---

## Часть 2: Delayed Messaging — отложенные сообщения

В RabbitMQ нет встроенной задержки сообщений по времени, но есть **плагин Delayed Message Plugin**.

---

### Альтернатива без плагинов: TTL + Dead Letter Exchange

**Сценарий**: отложить задачу на 30 секунд.

1. Создаёте **delay-queue** с аргументами:

   * `x-message-ttl: 30000`
   * `x-dead-letter-exchange: main_exchange`

2. Producer кладёт сообщение в delay-queue.

3. По истечении TTL сообщение **попадает в основную очередь**.

---

### Пример:


In [ ]:
channel.queue_declare(
    queue='delay_queue',
    arguments={
        'x-message-ttl': 30000,
        'x-dead-letter-exchange': '',
        'x-dead-letter-routing-key': 'main_queue'
    }
)



> После 30 секунд сообщение перейдёт в `main_queue`.

---



## Часть 3: Retry-паттерны

**Проблема**: сообщение не обработалось из-за временного сбоя (например, сервис базы данных упал).

---

### Решения:

1. **Повторная доставка (Nack)**:

   * Consumer вызывает `basic_nack(requeue=True)`.
   * Сообщение возвращается в очередь.
   * **Минус**: может создавать бесконечный цикл.

2. **Retry Queue с задержкой**:

   * При ошибке сообщение направляется в очередь с TTL.
   * После TTL возвращается в основную очередь.

**Это надёжный способ делать "отложенный ретрай".**

---



## Часть 4: Интеграция с Flask/FastAPI

RabbitMQ отлично сочетается с Python web-приложениями. Основная схема:

* Flask/FastAPI принимает HTTP-запрос.
* Создаёт сообщение и кладёт его в очередь.
* Worker (например, Celery или самостоятельный скрипт) забирает задачу и выполняет.

---

### Пример: Flask + RabbitMQ

#### Flask-приложение:


In [ ]:
# app.py
from flask import Flask, request
import pika

app = Flask(__name__)

connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()
channel.queue_declare(queue='tasks', durable=True)

@app.route('/send_task', methods=['POST'])
def send_task():
    data = request.json
    channel.basic_publish(
        exchange='',
        routing_key='tasks',
        body=str(data),
        properties=pika.BasicProperties(delivery_mode=2)
    )
    return {'status': 'Task queued'}

if __name__ == "__main__":
    app.run(port=5000)


Запускаете Flask и отправляете запрос:

```bash
curl -X POST -H "Content-Type: application/json" -d '{"task": "process data"}' http://localhost:5000/send_task
```

---



## Часть 5: Безопасность RabbitMQ

**По умолчанию:**

* Пользователь `guest` работает только с localhost.
* Все соединения нешифрованные.

---

### Как добавить пользователя:

```bash
rabbitmqctl add_user myuser mypassword
rabbitmqctl set_permissions -p / myuser ".*" ".*" ".*"
```

---

### Включение SSL (по желанию):

RabbitMQ может использовать TLS-сертификаты для шифрования соединений.

---



## Практическое задание:

1. Реализуйте RPC-сервис, который возвращает квадрат числа.
2. Сделайте очередь `delayed_tasks` с TTL=20 секунд и DLX.
3. Напишите consumer, который будет получать эти отложенные задачи.
4. Подключите Flask или FastAPI, чтобы класть задачи в очередь.
5. Добавьте нового пользователя и подключитесь под ним.




---

## 🧠 Краткий итог:

| Изучили                    | Зачем нужно                     |
| -------------------------- | ------------------------------- |
| RPC-паттерн                | Запрос–ответ через очередь      |
| Отложенные сообщения       | Планирование задач на будущее   |
| Retry через DLX            | Безопасный повтор обработки     |
| Интеграция с Flask/FastAPI | Связь HTTP-запросов с очередями |
| Аутентификация             | Защита брокера сообщений        |

---



---

# **Модуль 5. Мониторинг и отладка RabbitMQ**

---

## Цели занятия:

1. Изучить **интерфейс управления RabbitMQ** и его вкладки.
2. Познакомиться с **инструментами командной строки**.
3. Подключить мониторинг через **Prometheus и Grafana**.
4. Научиться диагностировать **зависшие сообщения** и "захлебнувшиеся" очереди.
5. Рассмотреть **стратегии поддержки и алертинга**.

---



## Часть 1: Веб-интерфейс RabbitMQ Management UI

> Адрес: [http://localhost:15672](http://localhost:15672)
> (по умолчанию логин/пароль: `guest/guest`)

### Вкладки:

* **Overview** — общая загрузка, соединения, сообщения
* **Connections** — все активные TCP-соединения
* **Channels** — логические каналы внутри соединений
* **Queues** — список всех очередей, их глубина и статус
* **Exchanges** — маршрутизаторы
* **Admin** — пользователи, разрешения, виртуальные хосты

---

### Что смотреть в первую очередь:

| Показатель                     | Что означает                                             |
| ------------------------------ | -------------------------------------------------------- |
| **Messages (Ready / Unacked)** | Сколько сообщений в очереди и сколько обрабатываются     |
| **Consumers**                  | Количество подписанных обработчиков                      |
| **Message rates (in / out)**   | Скорость поступления и обработки сообщений               |
| **Memory / Disk Alarms**       | RabbitMQ приостанавливает операции при нехватке ресурсов |

---



## Часть 2: Инструменты командной строки

### Базовые команды:

```bash
# Список очередей
rabbitmqctl list_queues

# Информация о соединениях
rabbitmqctl list_connections

# Статистика брокера
rabbitmqctl status

# Удалить очередь
rabbitmqctl delete_queue имя
```

### Диагностика:

```bash
rabbitmq-diagnostics -q memory_breakdown
rabbitmq-diagnostics check_running
rabbitmq-diagnostics check_port_connectivity
```

---



## Часть 3: Prometheus + Grafana мониторинг

> RabbitMQ умеет экспортировать метрики в Prometheus-совместимом формате. Для этого используется **плагин `rabbitmq_prometheus`**.

---

### Шаги подключения:

#### 1. Убедитесь, что включён плагин:

```bash
rabbitmq-plugins enable rabbitmq_prometheus
```

#### 2. Доступ к метрикам:

Откройте в браузере:
[http://localhost:15692/metrics](http://localhost:15692/metrics)

---

### Использование Docker Compose (пример):

```yaml
version: '3'
services:
  rabbitmq:
    image: rabbitmq:3-management
    ports:
      - "5672:5672"
      - "15672:15672"
      - "15692:15692"
    environment:
      - RABBITMQ_SERVER_ADDITIONAL_ERL_ARGS=-rabbit prometheus.tcp.port=15692
    volumes:
      - rabbitmq_data:/var/lib/rabbitmq
    networks:
      - monitoring

  prometheus:
    image: prom/prometheus
    volumes:
      - ./prometheus.yml:/etc/prometheus/prometheus.yml
    ports:
      - "9090:9090"
    networks:
      - monitoring

  grafana:
    image: grafana/grafana
    ports:
      - "3000:3000"
    networks:
      - monitoring

volumes:
  rabbitmq_data:

networks:
  monitoring:
```

#### 3. Конфигурация `prometheus.yml`:

```yaml
scrape_configs:
  - job_name: 'rabbitmq'
    static_configs:
      - targets: ['rabbitmq:15692']
```

---

### 📈 Grafana:

1. Перейдите на [http://localhost:3000](http://localhost:3000)
2. Войдите (admin/admin)
3. Добавьте источник данных Prometheus ([http://prometheus:9090](http://prometheus:9090))
4. Импортируйте дашборд RabbitMQ, например ID `10991` (готовый шаблон)

---



## Часть 4: Как отлаживать проблемы

### Проблема: Очередь переполняется

* Смотрим на вкладку **Queues → Messages Ready**
* Проверяем: есть ли consumer? работает ли он?

### Проблема: Сообщения не подтверждаются

* **Messages Unacked** растёт
* Возможные причины:

  * Consumer завис/сломался
  * Нет `basic_ack`

### Проблема: Высокая загрузка

* Проверяем **Channels**: не держат ли они "подвешенные" соединения
* Проверяем **Connections** на zombie-подключения

---



## Часть 5: Настройка алертов и лимитов

RabbitMQ сам может приостанавливать работу в случае:

* Недостатка памяти (memory alarm)
* Недостатка диска (disk alarm)

### Конфигурация в `rabbitmq.conf`:

```ini
vm_memory_high_watermark.relative = 0.4
disk_free_limit.absolute = 1GB
```

> Это означает: при превышении 40% ОЗУ или если меньше 1ГБ на диске — брокер остановит приём сообщений.

---



## Практическое задание:

1. Включите плагин `rabbitmq_prometheus`.
2. Поднимите `Prometheus` и `Grafana` в Docker Compose.
3. Импортируйте дашборд с ID `10991`.
4. Создайте очередь и отправьте 100 сообщений без consumer'а. Найдите очередь в графиках.
5. Подключите consumer — проверьте, как уменьшается очередь.
6. Откройте `rabbitmqctl list_queues` и сравните с UI.
7. Вызовите `rabbitmq-diagnostics check_port_connectivity` и интерпретируйте результат.

---



## Краткий итог:

| Что изучили                       | Почему это важно                           |
| --------------------------------- | ------------------------------------------ |
| Веб-интерфейс и его вкладки       | Видим состояние брокера в реальном времени |
| Команды CLI                       | Отладка без UI                             |
| Prometheus + Grafana              | Централизованный мониторинг                |
| Диагностика очередей и соединений | Устранение "зависших" сообщений            |
| Настройка лимитов ресурсов        | Предотвращение краха системы               |

---


---

# Шпаргалка по RabbitMQ

---

## Основные понятия

| Термин                   | Описание                         |
| ------------------------ | -------------------------------- |
| **Producer**             | Отправляет сообщения             |
| **Consumer**             | Получает сообщения               |
| **Queue**                | Очередь сообщений                |
| **Exchange**             | Маршрутизирует сообщения         |
| **Binding**              | Связь между очередью и exchange  |
| **Routing Key**          | Строка маршрутизации             |
| **Virtual Host (vhost)** | Изолированное пространство имен  |
| **Acknowledgment (ACK)** | Подтверждение доставки сообщения |

---



## Типы exchange

| Тип       | Поведение                                 |
| --------- | ----------------------------------------- |
| `direct`  | Один в один по routing key                |
| `fanout`  | Всем очередям, связанным с exchange       |
| `topic`   | Маршрутизация по шаблону (`.` и `*`, `#`) |
| `headers` | По заголовкам, а не routing key           |

---



## Структура взаимодействия

```plaintext
Producer → Exchange → [Binding] → Queue → Consumer
```

---


## Пример кода (Python pika)


In [ ]:
import pika

# Подключение
connection = pika.BlockingConnection(pika.ConnectionParameters("localhost"))
channel = connection.channel()

# Создание очереди
channel.queue_declare(queue="hello")

# Отправка
channel.basic_publish(exchange="", routing_key="hello", body="Привет!")

# Приём
def callback(ch, method, properties, body):
    print("Получено:", body)

channel.basic_consume(queue="hello", on_message_callback=callback, auto_ack=True)
channel.start_consuming()


---

## Команды CLI

```bash
# Общие
rabbitmqctl status                # Статус
rabbitmqctl list_queues          # Список очередей
rabbitmqctl list_connections     # Активные соединения
rabbitmqctl list_bindings        # Связи

# Управление очередями
rabbitmqctl purge_queue имя
rabbitmqctl delete_queue имя

# Пользователи
rabbitmqctl add_user login pass
rabbitmqctl set_user_tags login administrator
rabbitmqctl set_permissions -p / login ".*" ".*" ".*"

# Плагины
rabbitmq-plugins enable rabbitmq_management
rabbitmq-plugins enable rabbitmq_prometheus
```

---



## Конфигурация `rabbitmq.conf`

```ini
# Основные параметры
listeners.tcp.default = 5672
management.listener.port = 15672

# Ограничения ресурсов
vm_memory_high_watermark.relative = 0.4
disk_free_limit.absolute = 1GB
```

---



## Полезные порты

| Назначение         | Порт  |
| ------------------ | ----- |
| AMQP               | 5672  |
| HTTP UI            | 15672 |
| Prometheus метрики | 15692 |
| AMQPS (SSL)        | 5671  |

---

## ACK / NACK

| Действие                       | Метод                                     |
| ------------------------------ | ----------------------------------------- |
| Подтверждение                  | `basic_ack(delivery_tag)`                 |
| Отказ (без повторной доставки) | `basic_nack(delivery_tag, requeue=False)` |
| Отказ (с повтором)             | `basic_nack(delivery_tag, requeue=True)`  |

---



## Повторная доставка / Retry

1. Использовать `dead-letter exchange (DLX)`
2. Создать очередь с `x-dead-letter-exchange`
3. Отдельная очередь-задержка (`x-message-ttl` + DLX)

---



## Мониторинг

* Web UI: [http://localhost:15672](http://localhost:15672)
* CLI: `rabbitmqctl list_queues`
* Prometheus-плагин: `rabbitmq_prometheus`
* Grafana Dashboard: ID `10991`

---



## Стратегии

| Что делать...                   | Инструмент                           |
| ------------------------------- | ------------------------------------ |
| ...если сообщения копятся       | Проверить `Messages Ready`, consumer |
| ...если сообщения зависли       | Смотрим `Messages Unacked`           |
| ...если нужно масштабировать    | Добавить consumer'ов                 |
| ...если нужно надёжную доставку | Использовать `ack` + `durable`       |
| ...если очереди переполняются   | Включить лимиты памяти/диска         |
| ...если нужно повторять задачи  | DLX + TTL + retry queue              |
| ...если брокер умер             | Восстановить из бэкапа или кластер   |

---



## Проверка подключения и статуса

```bash
rabbitmq-diagnostics check_port_connectivity
rabbitmq-diagnostics check_running
rabbitmq-diagnostics -q memory_breakdown
```

---



## Docker Compose (базовая конфигурация)

```yaml
version: '3'
services:
  rabbitmq:
    image: rabbitmq:3-management
    ports:
      - "5672:5672"
      - "15672:15672"
```

---
